In [1]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.infogram import H2OInfogram
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule
import EvaluationModule_H2O
pd.options.mode.chained_assignment = None

(CVXPY) Jul 25 12:05:34 AM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 25 12:05:34 AM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 25 12:05:34 AM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
dataset = CoronnaCERTAINDataset(
    library_root = '/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge = 'regression_delta_binary', #option: regression, regression_delta, classification, binary_classification, regression_delta_binary
    dataset = 'CORRONA CERTAIN', 
    process_approach = 'SC', #option: KVB, SC
    imputation = 'IterativeImputer', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group = ['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group = 'all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points = (0,3), 
    train_test_rate = 0.8,
    remove_low_DAS = True,
    save_csv = False, 
    random_state = 2022,
    verbose=False)

In [4]:
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [5]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
# train_h2o = h2o.upload_file(str(train_loc))
# test_h2o = h2o.upload_file(str(test_loc))
train_h2o = h2o.import_file(str(train_loc))
test_h2o = h2o.import_file(str(test_loc))

# Identify predictors and response
x = train_h2o.columns[:-1]
# y = "DAS28_CRP_3M"
y = dataset.target

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13.0.2" 2020-01-14; Java(TM) SE Runtime Environment (build 13.0.2+8); Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from /Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmp7_gc6f3j
  JVM stdout: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmp7_gc6f3j/h2o_gaskell_started_from_python.out
  JVM stderr: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmp7_gc6f3j/h2o_gaskell_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_gaskell_eq645x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
for feature in dataset.categorical:
    train_h2o[feature] = train_h2o[feature].asfactor()
    test_h2o[feature] = test_h2o[feature].asfactor()
if "classification" in dataset.challenge:
    train_h2o[y] = train_h2o[y].asfactor()
    test_h2o[y] = test_h2o[y].asfactor()

In [7]:
# Run AutoML for 20 base models
# aml = H2OAutoML(max_models = 100, nfolds=10, balance_classes=True, max_after_balance_size=2.0, seed = 2022, project_name = "SC_classification_Jul24")
aml = H2OAutoML(max_models = 100, nfolds=10, seed = 2022, project_name = "SC_classification_Jul24_1")
aml.train(x=x, y=y, training_frame=train_h2o)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |
00:05:58.478: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]

█
00:06:03.427: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]

██
00:06:07.983: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]

█
00:06:11.610: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]
00:06:12.729: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]

██
00:06:17.583: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]

█
00:06:19.389: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]
00:06:20.446: _train param, Dropping bad and constant columns: [pres_imuran, pres_minocin, init_

H2OJobCancelled: Job<$03017f00000132d4ffffffff$_874af211c51088af62659e6b8d8d49d9> was cancelled by the user.

In [10]:
# lb.as_data_frame().sort_values(by=['mse'], ascending=True).head(20)

,model_id,rmse,mse,mae,rmsle,mean_residual_deviance
0,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.158155,1.341323,0.978010,NaN,1.341323
1,GBM_3_AutoML_1_20220724_222904,1.166049,1.359671,0.955289,NaN,1.359671
2,DeepLearning_grid_3_AutoML_1_20220724_222904_m...,1.168749,1.365974,0.971651,NaN,1.365974
3,DeepLearning_grid_3_AutoML_1_20220724_222904_m...,1.172850,1.375578,0.978555,NaN,1.375578
4,XGBoost_grid_1_AutoML_1_20220724_222904_model_2,1.182422,1.398122,0.996289,NaN,1.398122
5,StackedEnsemble_BestOfFamily_2_AutoML_1_202207...,1.182962,1.399399,0.991178,NaN,1.399399
6,StackedEnsemble_BestOfFamily_3_AutoML_1_202207...,1.183502,1.400677,0.982044,NaN,1.400677
7,StackedEnsemble_BestOfFamily_5_AutoML_1_202207...,1.183533,1.400751,0.986571,NaN,1.400751
8,StackedEnsemble_AllModels_4_AutoML_1_20220724_...,1.183998,1.401852,0.976739,NaN,1.401852
9,GBM_grid_1_AutoML_1_20220724_222904_model_7,1.184259,1.402469,0.970030,NaN,1.402469


In [11]:
# lb['model_id']

model_id
XGBoost_lr_search_selection_AutoML_1_20220724_222904_select_grid_model_1
GBM_3_AutoML_1_20220724_222904
DeepLearning_grid_3_AutoML_1_20220724_222904_model_2
DeepLearning_grid_3_AutoML_1_20220724_222904_model_1
XGBoost_grid_1_AutoML_1_20220724_222904_model_2
StackedEnsemble_BestOfFamily_2_AutoML_1_20220724_222904
StackedEnsemble_BestOfFamily_3_AutoML_1_20220724_222904
StackedEnsemble_BestOfFamily_5_AutoML_1_20220724_222904
StackedEnsemble_AllModels_4_AutoML_1_20220724_222904
GBM_grid_1_AutoML_1_20220724_222904_model_7


In [12]:
# lb.as_data_frame().sort_values(by=['mse'], ascending=True).loc[:30,"model_id"]

0     XGBoost_lr_search_selection_AutoML_1_20220724_...
1                        GBM_3_AutoML_1_20220724_222904
2     DeepLearning_grid_3_AutoML_1_20220724_222904_m...
3     DeepLearning_grid_3_AutoML_1_20220724_222904_m...
4       XGBoost_grid_1_AutoML_1_20220724_222904_model_2
5     StackedEnsemble_BestOfFamily_2_AutoML_1_202207...
6     StackedEnsemble_BestOfFamily_3_AutoML_1_202207...
7     StackedEnsemble_BestOfFamily_5_AutoML_1_202207...
8     StackedEnsemble_AllModels_4_AutoML_1_20220724_...
9           GBM_grid_1_AutoML_1_20220724_222904_model_7
10    StackedEnsemble_BestOfFamily_4_AutoML_1_202207...
11    StackedEnsemble_AllModels_3_AutoML_1_20220724_...
12    StackedEnsemble_AllModels_1_AutoML_1_20220724_...
13     XGBoost_grid_1_AutoML_1_20220724_222904_model_24
14                       GBM_1_AutoML_1_20220724_222904
15    StackedEnsemble_AllModels_2_AutoML_1_20220724_...
16          GBM_grid_1_AutoML_1_20220724_222904_model_9
17         GBM_grid_1_AutoML_1_20220724_222904_m

In [14]:
# aml_sc = EvaluationModule_H2O.AutoBuild(seed=1, project_name="EHR_RA_SC_July23_2", challenge=dataset.challenge, balance_class=1)

# i = 0
# for model_id in lb.as_data_frame().sort_values(by=['mse'], ascending=True).loc[:30,"model_id"]:
#     if "StackedEnsemble" not in model_id:
#         i+=1
#         model = h2o.get_model(model_id)
#         aml_sc.validate(model_id, model, train_h2o, test_h2o)
#     if i==10:
#         break

r_index: 100%|##########| 25/25 [00:00<00:00, 471.76it/s]


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [hxchf, pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf, hxtia]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


r_index: 100%|##########| 25/25 [00:00<00:00, 450.85it/s]


...

r_index: 100%|##########| 25/25 [00:00<00:00, 502.41it/s]

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [hxchf, pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [18]:
aml_sc.val_perf.head(50)

,model,MAE,MSE,RMSE,R2,Pearson_Correlation,Accuracy,F1-Score
0,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.184153,2.124602,1.457601,-0.857245,0.167644,0.677419,0.483333
1,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.003142,1.465549,1.210598,-0.544970,0.160431,0.677419,0.483333
2,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.097412,1.991282,1.411128,-0.535717,0.365706,0.741935,0.523077
3,XGBoost_lr_search_selection_AutoML_1_20220724_...,0.801877,1.124880,1.060604,0.006273,0.425941,0.774194,0.613191
4,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.234039,2.377902,1.542045,-0.079542,0.389040,0.766667,0.542484
5,XGBoost_lr_search_selection_AutoML_1_20220724_...,0.990813,1.551037,1.245406,-0.003600,0.284245,0.733333,0.423077
6,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.219195,2.323579,1.524329,-0.163113,0.336386,0.700000,0.558101
7,XGBoost_lr_search_selection_AutoML_1_20220724_...,0.949659,1.516285,1.231375,-0.490596,0.248274,0.700000,0.558101
8,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.045074,1.681426,1.296698,0.200638,0.458473,0.633333,0.387755
9,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.165989,1.857291,1.362825,0.078952,0.446501,0.733333,0.520000


In [19]:
aml_sc.test_perf.head(50)

,model,MAE,MSE,RMSE,R2,Pearson_Correlation,Accuracy,F1-Score
0,XGBoost_lr_search_selection_AutoML_1_20220724_...,1.124008,2.041361,1.428762,-0.466072,0.138056,0.658824,0.548617
1,GBM_3_AutoML_1_20220724_222904,1.058027,1.754860,1.324711,-0.260312,0.254500,0.705882,0.568177
2,DeepLearning_grid_3_AutoML_1_20220724_222904_m...,1.213572,2.305900,1.518519,-0.656059,0.305151,0.705882,0.584067
3,DeepLearning_grid_3_AutoML_1_20220724_222904_m...,1.303683,2.493627,1.579122,-0.790881,0.241903,0.717647,0.632035
4,XGBoost_grid_1_AutoML_1_20220724_222904_model_2,1.187237,2.254990,1.501662,-0.619496,0.131650,0.588235,0.498229
5,GBM_grid_1_AutoML_1_20220724_222904_model_7,1.154420,2.000824,1.414505,-0.436958,0.153888,0.658824,0.499086
6,XGBoost_grid_1_AutoML_1_20220724_222904_model_24,1.078260,1.721145,1.311924,-0.236098,0.197758,0.611765,0.501333
7,GBM_1_AutoML_1_20220724_222904,1.177760,2.072974,1.439783,-0.488775,0.196560,0.658824,0.478307
8,GBM_grid_1_AutoML_1_20220724_222904_model_9,1.194730,2.067464,1.437868,-0.484819,0.198071,0.682353,0.514286
9,GBM_grid_1_AutoML_1_20220724_222904_model_12,1.043396,1.760075,1.326678,-0.264057,0.292765,0.647059,0.491627


In [42]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_lr_search_selection_AutoML_1_20220724_222904_select_grid_model_1


Model Summary: 


,,number_of_trees
0,,31.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.47536937045789057
RMSE: 0.6894703550247034
MAE: 0.5362376985507772
RMSLE: NaN
Mean Residual Deviance: 0.47536937045789057

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 1.446102672573799
RMSE: 1.2025400918779379
MAE: 0.9737197342696354
RMSLE: NaN
Mean Residual Deviance: 1.446102672573799

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.973956,0.058759,0.995007,1.002539,0.912949,0.913448,1.045838
1,mean_residual_deviance,1.446900,0.165129,1.314796,1.541315,1.306257,1.382733,1.689401
2,mse,1.446900,0.165129,1.314796,1.541315,1.306257,1.382733,1.689401
3,r2,0.037673,0.092748,0.080473,0.140492,-0.068544,-0.052550,0.088496
4,residual_deviance,1.446900,0.165129,1.314796,1.541315,1.306257,1.382733,1.689401
5,rmse,1.201345,0.067737,1.146645,1.241497,1.142916,1.175896,1.299770
6,rmsle,0.640839,0.121023,0.536415,0.740084,NaN,0.751068,0.535789



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-07-24 22:30:04,0.177 sec,0.0,1.619102,1.247555,2.621492
1,,2022-07-24 22:30:04,0.186 sec,5.0,1.138718,0.884590,1.296678
2,,2022-07-24 22:30:04,0.195 sec,10.0,0.993756,0.787982,0.987551
3,,2022-07-24 22:30:04,0.202 sec,15.0,0.895574,0.710016,0.802052
4,,2022-07-24 22:30:04,0.209 sec,20.0,0.815720,0.634302,0.665400
5,,2022-07-24 22:30:04,0.216 sec,25.0,0.752178,0.586556,0.565772
6,,2022-07-24 22:30:04,0.233 sec,30.0,0.703268,0.545920,0.494585
7,,2022-07-24 22:30:04,0.237 sec,31.0,0.689470,0.536238,0.475369



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,DAS28_CRP_0M,41.874229,1.000000,0.134947
1,weight,21.723515,0.518780,0.070008
2,usresultsCRP,20.026340,0.478250,0.064538
3,seatedbp2,16.387033,0.391339,0.052810
4,BMI,15.445545,0.368856,0.049776
5,swollen_jts_28,14.792601,0.353263,0.047672
6,age,13.578554,0.324270,0.043759
7,usresultsRF,13.166860,0.314438,0.042432
8,tender_jts_28,12.188334,0.291070,0.039279
9,usresultsIgA,11.387392,0.271943,0.036698



See the whole table with table.as_data_frame()


In [49]:
m = h2o.get_model(lb.as_data_frame().sort_values(by=['mse'], ascending=True).loc[:30,"model_id"][1])

In [52]:
# dir(m)

In [21]:
# aml_sc.saved_model

In [ ]:
# aml_sc = evaluate.AutoBuild(seed=1, project_name="EHR_RA_SC")
# test, test_loc = dataset.get_test()
# baseline = test['DAS28_CRP_0M']
# true = test['DAS28_CRP_3M']

In [ ]:
# pred = pred.as_data_frame()

In [22]:
perf = aml.leader.model_performance(train_h2o)
perf


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.47536937045789057
RMSE: 0.6894703550247034
MAE: 0.5362376985507772
RMSLE: NaN
Mean Residual Deviance: 0.47536937045789057


In [23]:
perf = aml.leader.model_performance(test_h2o)
perf


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 1.341323021701554
RMSE: 1.158155007631342
MAE: 0.9780099946786376
RMSLE: NaN
Mean Residual Deviance: 1.341323021701554


In [28]:
# aml_sc.saved_model

In [29]:
# aml_sc.evaluate('XGBoost_lr_search_selection_AutoML_1_20220724_222904_select_grid_model_1', aml.leader, "test", testset, aml_sc.test_perf)

In [40]:
model = aml.leader
true = test[dataset.target]
X = test.drop(columns=dataset.target)
X = h2o.H2OFrame(X)
pred = model.predict(X).as_data_frame()['predict']
pred

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


0     1.255168
1     2.286309
2     2.392133
3     0.812751
4     1.259574
        ...   
80    1.178084
81    0.396744
82    1.674447
83    1.862879
84    1.558436
Name: predict, Length: 85, dtype: float64

In [41]:
from sklearn import metrics
metrics.mean_squared_error(true, pred)

1.3413230520336332

In [33]:
model.predict(test_h2o)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'race_grp' has levels not trained on: ["5"]
  warnings.warn(w)


predict
1.25517
2.28631
2.39213
0.812751
1.25957
1.08353
2.06492
1.15485
1.90849
1.48913


In [24]:
aml.leader.predict(test_h2o)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'race_grp' has levels not trained on: ["5"]
  warnings.warn(w)


predict
1.25517
2.28631
2.39213
0.812751
1.25957
1.08353
2.06492
1.15485
1.90849
1.48913


In [ ]:
# # Explain an AutoML object
# exa = aml.explain(test_h2o)